In [0]:
%pip install -e ..
%restart_python

In [0]:

#For databricks to capture git imported files path(not recommended for other things)
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent / 'src'))

In [0]:
pip install kagglehub

In [0]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohitbansal31s/marvel-characters")

print("Path to dataset files:", path)

In [0]:
import pandas as pd
import yaml
from loguru import logger
from pyspark.sql import SparkSession

from marvel_characters.config import ProjectConfig
from marvel_characters.data_processor import DataProcessor

config = ProjectConfig.from_yaml(config_path="../project_config_marvel.yml", env="dev")

logger.info("Configuration loaded:")
logger.info(yaml.dump(config, default_flow_style=False))

In [0]:
# Load the Marvel characters dataset
spark = SparkSession.builder.getOrCreate()

filepath = "/home/spark-22df7628-de02-481c-b589-2b/.cache/kagglehub/datasets/mohitbansal31s/marvel-characters/versions/1/marvel_characters_dataset.csv"

# Load the data
df = pd.read_csv(filepath)

# Display basic info about the dataset
logger.info(f"Dataset shape: {df.shape}")
logger.info(f"Columns: {list(df.columns)}")
logger.info(f"Target column '{config.target}' distribution:")
logger.info(df[config.target].value_counts())

In [0]:
df.head()

In [0]:
# Load the Marvel characters dataset

data_processor = DataProcessor(df, config, spark)

# Preprocess the data
data_processor.preprocess()

logger.info(f"Data preprocessing completed.")

In [0]:
# Split the data
X_train, X_test = data_processor.split_data()
logger.info("Training set shape: %s", X_train.shape)
logger.info("Test set shape: %s", X_test.shape)

In [0]:

# Save to catalog
logger.info("Saving data to catalog")
data_processor.save_to_catalog(X_train, X_test)

# Enable change data feed (only once!)
logger.info("Enable change data feed")
data_processor.enable_change_data_feed()